In [ ]:
# 필수 라이브러리 설치 (Colab)
!pip install -q --upgrade transformers peft accelerate datasets evaluate scikit-learn bitsandbytes

# GPU 정보 확인
!nvidia-smi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 130.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 42.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatibl

In [ ]:
import os, random, math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "monologg/koelectra-base-v3-discriminator"  # KoELECTRA 분류
OUTPUT_DIR = "/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/koelectra_lora_out"
MAX_LEN = 512            # 문단이 길면 384~512, 더 길면 슬라이딩 윈도우 사용할수도?
VAL_SIZE = 0.1           # 검증데이터셋 비율


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/train.csv', encoding='utf-8-sig')
test = pd.read_csv('/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/test.csv', encoding='utf-8-sig')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# 문자열 캐스팅 + 결측치 제거
df["full_text"] = df["full_text"].astype(str).fillna("")
df["title"] = df["title"].astype(str).fillna("")

# Stratified split (라벨 비율 유지)
train_df, valid_df = train_test_split(
    df[["full_text","generated"]],
    test_size=VAL_SIZE,
    random_state=SEED,
    stratify=df["generated"]
)

print(train_df.shape, valid_df.shape)
train_df.head()


(87454, 2) (9718, 2)


,full_text,generated
13353,"아메리칸 익스프레스 컴퍼니, 약칭 아멕스(""AmEx"", ""Amex"")는 미국계 다국...",0
5317,"장 담그기는 대한민국의 국가무형문화재 제137호이다. 장(醬)은 고추장, 된장, 간...",0
58382,이극감(李克堪)의 장남이자 이준경(李浚慶)의 조부이다. 무오사화 당시 김종직과 그의...,0
18605,"츠비카우는 독일 동부 작센주에 있는 도시이다. \n 작센 주 남부, 에르츠 산맥 기...",0
83834,김기홍은 대법원 판사를 역임한 법조인이다. 1981년 청주지방법원장에 임명된 오석낙...,0


In [ ]:
# MODEL_NAME = "monologg/koelectra-base-v3-discriminator"  # KoELECTRA 분류기
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 토크나이즈 함수
def encode_batch(batch):
    return tokenizer(
        batch["full_text"],
        truncation=True,
        padding=False,      # Trainer 내부 collator에서 pad 수행
        max_length=MAX_LEN
    )

# HuggingFace Datasets로 변환 (labels 컬럼명으로 교체 필요)
train_ds = Dataset.from_pandas(train_df.rename(columns={"generated":"labels"}), preserve_index=False)
valid_ds = Dataset.from_pandas(valid_df.rename(columns={"generated":"labels"}), preserve_index=False)

train_ds = train_ds.map(encode_batch, batched=True, remove_columns=["full_text"])
valid_ds = valid_ds.map(encode_batch, batched=True, remove_columns=["full_text"])

# 동적 패딩 collator (배치 내 최대 길이에 맞춰 pad)
collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/87454 [00:00<?, ? examples/s]

Map:   0%|          | 0/9718 [00:00<?, ? examples/s]

In [ ]:
# 분류 헤드 달린 ELECTRA 로드
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# LoRA 설정 (ELECTRA/BERT류: attention의 query/value에 얹는 게 안전)
lora_cfg = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,                 # 랭크 (작게→적은 VRAM / 너무 작으면 표현력↓)
    lora_alpha=16,       # 스케일
    lora_dropout=0.1,
    target_modules=["query","value"]  # 필요시 ["key","dense"] 추가도 가능
)

model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()  # LoRA 파라미터만 학습되는지 확인
model.to(device)


pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 887,042 || all params: 113,809,924 || trainable%: 0.7794


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): ElectraForSequenceClassification(
      (electra): ElectraModel(
        (embeddings): ElectraEmbeddings(
          (word_embeddings): Embedding(35000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): ElectraEncoder(
          (layer): ModuleList(
            (0-11): 12 x ElectraLayer(
              (attention): ElectraAttention(
                (self): ElectraSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
          

In [ ]:
# 라벨 비율에 기반한 가중치 (0/1 분포 불균형시 효과적)
neg = (train_df["generated"] == 0).sum()
pos = (train_df["generated"] == 1).sum()
pos_weight = neg / max(1, pos)   # 예: 11:1이면 ~11

class_weights = torch.tensor([1.0, pos_weight], dtype=torch.float)

# 클래스 가중치 (이미 계산했다고 가정)
# class_weights = torch.tensor([1.0, pos_weight], dtype=torch.float)

def weighted_loss(model, inputs, return_outputs=False, **kwargs):  # ← **kwargs 추가!
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fn(logits, labels.long())
    return (loss, outputs) if return_outputs else loss


In [ ]:
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")
metric_roc = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    import numpy as np
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
    preds = probs.argmax(axis=1)
    acc = metric_acc.compute(predictions=preds, references=labels)["accuracy"]
    f1  = metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    try:
        auc = metric_roc.compute(prediction_scores=probs[:,1], references=labels)["roc_auc"]
    except:
        auc = float("nan")
    return {"accuracy": acc, "f1_macro": f1, "auc": auc}


In [ ]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # ← **kwargs
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fn(logits, labels.long())
        return (loss, outputs) if return_outputs else loss

In [ ]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-4,                # LoRA만 학습하므로 base보다 조금 크게
    per_device_train_batch_size=32,    # GPU 여유에 맞게 16~64 사이 조정
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    warmup_ratio=0.05,
    weight_decay=0.01,
    logging_steps=200,
    eval_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),    # Colab T4/V100에서 Mixed Precision
    report_to="none",                  # (원하면 wandb 등으로 변경)
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

# 가중치 손실 적용
trainer.compute_loss = weighted_loss

trainer.train()


/tmp/ipython-input-4256498411.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1 Macro,Auc
1000,0.345500,0.372205,0.804178,0.650759,0.914324
2000,0.325600,0.307930,0.917370,0.786097,0.927282
3000,0.294300,0.290044,0.920148,0.794744,0.940174
4000,0.292100,0.296735,0.924882,0.805624,0.943065
5000,0.278400,0.323055,0.864684,0.721758,0.950261
6000,0.246500,0.303842,0.892056,0.757324,0.952194
7000,0.253100,0.252513,0.944433,0.845534,0.953361
8000,0.269800,0.309596,0.896584,0.764077,0.953974


TrainOutput(global_step=8199, training_loss=0.2956816790408951, metrics={'train_runtime': 7802.5993, 'train_samples_per_second': 33.625, 'train_steps_per_second': 1.051, 'total_flos': 6.974527732612301e+16, 'train_loss': 0.2956816790408951, 'epoch': 3.0})

In [ ]:
# 어댑터(LoRA)만 저장 → 베이스 모델 위에 merge 없이 로드 가능
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# (선택) LoRA를 베이스에 merge하고 싶은 경우:
# from peft import PeftModel
# merged = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
# merged = merged.merge_and_unload()     # 가중치 병합
# merged.save_pretrained(OUTPUT_DIR + "_merged")


('/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/koelectra_lora_out/tokenizer_config.json',
 '/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/koelectra_lora_out/special_tokens_map.json',
 '/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/koelectra_lora_out/vocab.txt',
 '/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/koelectra_lora_out/added_tokens.json',
 '/content/drive/MyDrive/LikeLion_NLP2/Small_Challenge/koelectra_lora_out/tokenizer.json')